# Launch your digital twin: Cloud harvester
This is a KWH notebook which is regarded as a run-time environment for a emulated asset of harvester. The corresponding digital twin lives in the S³I Repository, whose configurtion file was created with the notebook 08a and stored in the folder configs.The emulated asset will automatically connect to S³I and has **no S³I-B endpoint** to receive S³I-B messages.
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I and modelling language library.

In [ ]:
import s3i
import time 
import os 
import json
import ml
import logging
from tools import print_with_timestamp, yes, no

In [ ]:
%%html
<img src="harvester_images/folie_1_2.png", width=1000, height=1000>

## Load the configuration file 

In [ ]:
thing_name = input("[S³I]: Please enter the name of your harvester (my_cloud_harvester): ")
config_file_name = "{}.json".format(thing_name)
config_path = os.path.abspath(os.path.join("", "configs"))
thing_model = ml.load_config(config_filepath=os.path.join(config_path, config_file_name))
print_with_timestamp("The fml40 model of the created harvester in the form of JSON: " + json.dumps(thing_model, indent=2))

## Set up a logger

In [ ]:
ml.setup_logger(thing_model["attributes"].get("name", None))

## Build the digital twin
The construction of the digital twin is based on the configuration file created previously. Features, roles and another relevant entries, which are configured in the configuration file, will be instantiated with the resptive object of the fml40 library. 

In [ ]:
%%html
<img src="harvester_images/cloud_dt.png", width=1000, height=1000>

In [ ]:
cred_filepath = os.path.join(config_path, "{}_cred.json".format(thing_name))
with open(cred_filepath) as file:
    cred = json.load(file)
    
thing = ml.create_thing(
                     model=thing_model, grant_type="client_credentials", 
                     secret=cred.get("secret"),
                     is_broker=False, is_repo=True
                    )

We offer hereby a pre-defined simulate function, which simulates the rpm changing of harvester. The current rpm will be uploaded into the repo entry of the digital twin stored in the S³I Repository

In [ ]:
def emulate_rpm():
    """
    This function emulates how the rpm value of the engine changes between 2000 to 2500 U/min. 
    """
    tank = "up"

    while True:
        if tank == "down":
            __new_rpm = thing.features["ml40::RotationalSpeed"].rpm - 10
            if __new_rpm < 2000:
                tank = "up"

        elif tank == "up":
            __new_rpm = thing.features["ml40::RotationalSpeed"].rpm + 10
            if __new_rpm > 2500:
                tank = "down"
        thing.features["ml40::RotationalSpeed"].rpm = __new_rpm
        time.sleep(1)

Insert the class and function into the instance of the asset.

In [ ]:
thing.add_user_def(func=emulate_rpm)

## Launch the thing
The thing is then started and connect to S³I automatically. It will firstly authenticate itself at the S³I IdentityProvider. The thing has a directory entry in the S³I Directory and a repository entry stored in the S³I Repository, which is synchronized with the emulated rpm value.

In [ ]:
thing.run_forever()
while True:
    i = input("[S³I]: End the debug outputs? (j/n)")
    if i in yes:
        break
    elif i in no:
        continue